In [536]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd

import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

import time

In [537]:
html_doc = requests.get('https://www.foxnews.com/politics')
soup = BeautifulSoup(html_doc.text, 'html.parser')

In [538]:
chrome_path = "./chromedriver"
options = webdriver.ChromeOptions()
options.add_argument('--load-extension=./adblock4Sellenium/3.34.0_0')
#options.add_experimental_option( "prefs",{'profile.managed_default_content_settings.javascript': 2})
#options.add_argument("--user-agent=" + 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36')
driver = webdriver.Chrome(chrome_path, options=options)

In [539]:
def filterWords(words):
    newWords = []
    for word in words:
        word = word.lower()
        word = word.replace("!", "")
        word = word.replace("?", "")
        word = word.replace(":", "")
        word = word.replace(";", "")
        word = word.replace(".", "")
        word = word.replace(",", "")
        word = word.replace('\'', "")
        word = word.replace('\"', "")
        #words below are ones where we want to connect together to ensure these words are joined
        word = word.replace("trumps", "trump")
        word = word.replace("russian", "russia")
        word = word.replace("russians", "russia")
        word = word.replace("dems", "democrat")
        word = word.replace("democrats", "democrat")
        word = word.replace("republicans", "republican")
        word = word.replace("libs", "liberal")
        word = word.replace("liberals", "liberal")
        word = word.replace("conservatives", "conservative")
        word = word.replace("taxes", "tax")
        word = word.replace("socialists", "socialist")
        word = word.replace("nationalists", "nationalist")
        word = word.replace("globalists", "globalist")
        word = word.replace("corrupted", "corrupt")
        newWords.append(word)
    return newWords

In [540]:
def collect(soup):
    articlesDF = []
    wordsDF = []
    articleList = soup.find('section', {'class': "collection collection-article-list has-load-more"})
    articles = articleList.findAll('article')
    print("articles length: ", len(articles))
    #TODO click on see more
    for article in articles:
        linkHTML = article.find('a')
        articleTime = article.find('span', {'class': 'time'}).text
        if 'minutes' in articleTime or 'minute' in articleTime:
            continue
        if 'hour' in articleTime or 'hours' in articleTime:
            if '9' in articleTime or '8' in articleTime or '7' in articleTime or '6' in articleTime or '5' in articleTime  or '4' in articleTime  or '3' in articleTime or '2' in articleTime or '1' in articleTime:
                continue
        link = linkHTML['href']
        if 'politics' in link:
            try:
                url = 'https://www.foxnews.com' + link
                driver.get(url)
                #print(driver.current_url)
                #htmlArticle = requests.get('https://www.foxnews.com/' + link)
                soupArticle = BeautifulSoup(driver.page_source, 'html.parser')
                #get article title

                #words in title
                header = soupArticle.find('header', {'class': 'article-header'})
                title = header.find('h1').text
                print(title)
                wordsInTitle = title.split()
                wordsInTitle = filterWords(wordsInTitle)

                getIcons = driver.find_element_by_class_name('article-social')
                getIcons = getIcons.find_element_by_class_name('social-icons')
                commentButton = getIcons.find_element_by_class_name('comments')
                #commentButton = commentButton.find_element_by_tag_name('a')

                #IT DOESNT CLICK
                try:
                    commentButton.click()
                    time.sleep(10)

                    iframe = driver.find_element_by_class_name('sppre_frame-container')
                    iframe = iframe.find_element_by_tag_name('iframe')
                    #iframe = driver.find_elements_by_id('bb4b0ab3b34f383d8da32a70bad542e1-iframe')
                    driver.switch_to.frame(iframe)
                    newSoupArticle = BeautifulSoup(driver.page_source, 'html.parser')
                    #wrong >:(
                    commentCount = newSoupArticle.find('span', {'class': 'param-messagesCount'}).text
                    commentCount = commentCount.replace(",", "")
                    #print(commentCount)
                    commentCount = int(commentCount)
                    articlesDF.append({'url': url, 'articleTitle': title,
                                    'commentCount': commentCount})
                    for word in wordsInTitle:

                        wordsDF.append({'url': url, 'articleTitle': title, 'word': word,
                                        'commentCount': commentCount})
                except:
                    print("CANNOT GET COMMENT COUNT")
            except:
                print("ERROR BEFORE GETTING TO COMMENTS")

    return [articlesDF, wordsDF]

In [541]:
driver.get('https://www.foxnews.com/politics')

for x in range(1, 50):
    try:
        loadMore = driver.find_element_by_class_name('load-more')
        loadMore.click()
    except:
        print("cant click load more")
    time.sleep(.2)
time.sleep(5)


soup = BeautifulSoup(driver.page_source, 'html.parser')
dfs = collect(soup)


articles length:  465
Comey: Trump is lucky a sitting president can’t be indicted for being linked to Cohen case
Nick Ayers, considered candidate for White House chief of staff, 'departing' at year's end
Bob Bergland, former agriculture secretary, dies at 90
Schiff: Dems 'hope to bring' Cohen back to testify on possible collusion, obstruction of justice
Graham: Saudis would 'be 'speaking Farsi in about a week' without US support against Iran
Lame-duck session sees GOP leaders rushing to pass border wall, criminal justice reform
Huawei executive's arrest will not hinder trade talks with China, Kudlow says
Trump campaign finance violation would 'certainly' be impeachable offense, top Dem says
Rubio: Trump pardoning Manafort would be a 'terrible mistake'
Trump blasts 'Leakin' James Comey' following former FBI chief's testimony on Capitol Hill
Comey reveals he concealed Trump meeting memo from DOJ leaders
Comey says Trump wasn't among 'four Americans' targeted in FBI probe
Comey transcript

George H.W. Bush remembered as someone ‘you would’ve wanted on your side,’ Sen. Alan Simpson eulogizes
George W. Bush reveals George H.W. Bush's 'last words on Earth' during emotional eulogy
George H.W. Bush's biographer Jon Meacham draws laughs from Bush kids during eulogy
George W. Bush appears to hand Michelle Obama something at father's funeral, recreating viral moment
CANNOT GET COMMENT COUNT
At George H.W. Bush's funeral, Trump, Clintons don't shake hands
CANNOT GET COMMENT COUNT
George H.W. Bush honored by son, dignitaries in emotional funeral tribute: 'A great and noble man'
CANNOT GET COMMENT COUNT
George H.W. Bush's funeral program for Wednesday's service at Washington National Cathedral
George H.W. Bush once shaved head to support Secret Service agent's sick son 'in classic 41 manner'
How Avenatti’s 2020 aspirations fizzled: ‘Skeletons came to light’
Despite heat at home, Trump approval ratings tower above embattled European leaders’
George H.W. Bush funeral will be attended

In [543]:
articlesDF = pd.DataFrame(dfs[0], columns = ['url', 'articleTitle', 'commentCount'])
wordsDF = pd.DataFrame(dfs[1], columns = ['url', 'articleTitle', 'word', 'commentCount'])

In [665]:
#removing words that have no politial connection (filler words) - scores are most likely coincidental
wordsDF = wordsDF[wordsDF.word != 'to']
wordsDF = wordsDF[wordsDF.word != 'in']
wordsDF = wordsDF[wordsDF.word != 'for']
wordsDF = wordsDF[wordsDF.word != 'of']
wordsDF = wordsDF[wordsDF.word != 'on']
wordsDF = wordsDF[wordsDF.word != 'a']
wordsDF = wordsDF[wordsDF.word != 'at']
wordsDF = wordsDF[wordsDF.word != 'be']
wordsDF = wordsDF[wordsDF.word != 'the']
wordsDF = wordsDF[wordsDF.word != 'with']
wordsDF = wordsDF[wordsDF.word != 'by']
wordsDF = wordsDF[wordsDF.word != 'from']
wordsDF = wordsDF[wordsDF.word != 'is']
wordsDF = wordsDF[wordsDF.word != 'about']
wordsDF = wordsDF[wordsDF.word != 'as']
wordsDF = wordsDF[wordsDF.word != 'over']
wordsDF = wordsDF[wordsDF.word != 'team']
wordsDF = wordsDF[wordsDF.word != 'out']
wordsDF = wordsDF[wordsDF.word != 'they']
wordsDF = wordsDF[wordsDF.word != 'that']
wordsDF = wordsDF[wordsDF.word != 'over']
wordsDF = wordsDF[wordsDF.word != 'end']
wordsDF = wordsDF[wordsDF.word != 'wants']
wordsDF = wordsDF[wordsDF.word != 'called']
wordsDF = wordsDF[wordsDF.word != 'calls']
wordsDF = wordsDF[wordsDF.word != 'over']
wordsDF = wordsDF[wordsDF.word != '$$']
wordsDF = wordsDF[wordsDF.word != 'foundation']
wordsDF = wordsDF[wordsDF.word != 'have']
wordsDF = wordsDF[wordsDF.word != '9th']
wordsDF = wordsDF[wordsDF.word != 'into']
wordsDF = wordsDF[wordsDF.word != 'after']
wordsDF = wordsDF[wordsDF.word != 'key']
wordsDF = wordsDF[wordsDF.word != 'among']
wordsDF = wordsDF[wordsDF.word != 'could']
wordsDF = wordsDF[wordsDF.word != 'says']
wordsDF = wordsDF[wordsDF.word != 'has']
wordsDF = wordsDF[wordsDF.word != 'him']

#it wont let me name it anything else
wordCounts = wordsDF['word'].value_counts()
wordCountsDF = pd.DataFrame(wordCounts, columns= ['word'])


#for word in words


In [662]:
averageCommentsPerWord = wordsDF.groupby('word', as_index=False)['commentCount'].mean()
only1 = wordCounts[wordCounts == 1]
for word in only1.index:
    averageCommentsPerWord = averageCommentsPerWord[averageCommentsPerWord.word != word]
        
averageCommentsPerWord
averageCommentsPerWord = averageCommentsPerWord.sort_values('commentCount', ascending=False)
articlesDF = articlesDF.sort_values('commentCount', ascending=False)

In [666]:
articlesDF.to_csv('foxArticlesCommentCount.csv', index = False)
wordCounts.to_csv('foxWordCounts.csv', index = False)
averageCommentsPerWord.to_csv('foxWordAvgCommentCount.csv', index = False)
